In [2]:
# encoding: UTF-8
from Queue import Queue, Empty
from sine.threads import ReStartableThread

class EventManager:
    '''
    以 key 为键标识每个事件，对一个 key 可以添加或移除多个监听器。
    发布事件时传入 key 和数据 data ，后者将会传给监听器。
    '''
    def __init__(self):
        self._eventQueue = Queue()
        self._listenThread = ReStartableThread(target = self._run)
        # 保存对应的事件的响应函数，每个键对应一个集合
        self._map = {}

    def _run(self, stop_event):
        while True:
            try:
                event = self._eventQueue.get(block = True, timeout = 0.1)
                if stop_event.is_set():
                    return
                self._process(event)
            except Empty:
                pass

    def _process(self, event):
        try:
            for listener in self._map[event[0]]:
                listener(event[1])
        except KeyError:
            pass

    def start(self):
        '''开始事件监听。'''
        self._listenThread.start()

    def stop(self):
        '''停止事件监听。'''
        self._listenThread.stop()
    
    def clear(self):
        '''清空事件。'''
        self._eventQueue.queue.clear()

    def addListener(self, key, listener):
        self._map.setdefault(key, set()).add(listener)
            
    def removeListener(self, key, listener):
        try:
            self._map[key].remove(listener)
        except KeyError:
            pass
        
    def sendEvent(self, key, data=None):
        '''发布事件。以 key 标识；data 为数据，将会传给监听器。'''
        self._eventQueue.put((key, data))
